In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn import datasets
import pandas as pd 

In [2]:
datas = datasets.load_breast_cancer()
X, Y = datas.data, datas.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 3119)

# df_breast_cancer = pd.DataFrame(datas.data, columns = datas.feature_names)
# df_breast_cancer["target"] = datas.target

# df_breast_cancer

In [3]:
class Node:
    def __init__(self, feature = None, threshold = None, left = None, right = None,*, value = None):
        self.feature = feature
        self.right = right
        self.left = left
        self.threshold = threshold
        self.value = value
    
    def is_leaf(self):
        return self.value is not None

In [4]:
class decision_tree:
    def __init__(self, max_depth = 100, min_samples_split = 2, n_features = None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.root = None


    def fit(self, X, Y): 
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, Y)


    def _grow_tree(self, X, Y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(Y))

        #terminate condition
        if  (depth >= self.max_depth or n_samples < self.min_samples_split or n_labels == 1) :
            leaf_value = self._most_common_label(Y)
            return Node(value = leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        #best split
        best_feature, best_threshold = self._best_split(X, Y, feat_idxs)

        #create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        left = self._grow_tree(X[left_idxs, :], Y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], Y[right_idxs], depth+1)
        return Node(best_feature, best_threshold, left, right)

    def _most_common_label(self, Y):
        counter = Counter(Y)
        return counter.most_common(1)[0][0]
    
    def _best_split(self, X, Y, feat_idxs):
        
        best_gain = -1
        best_feature, best_threshold = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                #compute information gain
                gain = self._information_gain(X_column, Y, thr)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feat_idx
                    best_threshold = thr 
        return best_feature, best_threshold


    def _information_gain(self, X_column, Y, threshold):
        #parent entropy
        parent_entropy = self._entropy(Y)

        #create children 
        left_idxs, right_idxs = self._split(X_column, threshold)
        n_lefts  = len( left_idxs)
        n_rights = len(right_idxs)

        if ( n_lefts == 0 or n_rights ==0):
            return 0

        #weighted children entropy
        left_child_entropy  = self._entropy(Y[left_idxs])
        right_child_entropy = self._entropy(Y[right_idxs])

        weighted_children_entropy = (n_lefts/len(Y)) * left_child_entropy + (n_rights/len(Y)) * right_child_entropy

        #information gain
        IG = parent_entropy - weighted_children_entropy

        return IG
    
    def _entropy(self, Y):
        hist = np.bincount(Y)
        Py = hist/len(Y)

        return -np.sum([p * np.log(p) for p in Py if p > 0])
    
    def _split(self, X_column, threshold):
        left_idxs  = np.argwhere(X_column <= threshold).flatten()
        right_idxs = np.argwhere(X_column > threshold).flatten()

        return left_idxs, right_idxs


    def predict(self, X_test):
        return np.array([self._traverse_tree(x, self.root) for x in X_test])
    
    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    

In [8]:
clf = decision_tree()
clf.fit(X_train, Y_train)

predictions = clf.predict(X_test)

def Accuracy(Y_test, predictions):
    return np.sum(Y_test == predictions)/len(Y_test)

print(f"Accuracy: {round(Accuracy(Y_test, predictions), 2)*100}%")

Accuracy: 96.0%
